In [ ]:
!git clone --depth 1 https://github.com/roguehunter7/CornerSentinal

Cloning into 'CornerSentinal'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 42 (delta 1), reused 39 (delta 0), pack-reused 0
Receiving objects: 100% (42/42), 247.50 MiB | 9.96 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (39/39), done.


In [ ]:
!pip install ultralytics filterpy scikit-image lap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=6597ad474f7f868c6ef39641df0a64a8e32a5a6682f8f1101d44bbcec37dc683
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628964 sha256=4b733fc55637b32e604076d32625143403698464d80859b8025031629cd460a8
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built filterpy lap


In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8x.pt')

# Open the video file
video_path = "/content/CornerSentinal/test_images/rightside.mp4"
cap = cv2.VideoCapture(video_path)

# Define the constants for speed calculation
VIDEO_FPS = 30
FACTOR_KM = 3.6
LATENCY_FPS = 15

# Store the track history and speed information
track_history = defaultdict(lambda: [])

# Define the output video file path with MP4 format
output_video_path = "/content/CornerSentinal/test_images/rightside_out.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use "mp4v" for H.264 compression
out = cv2.VideoWriter(output_video_path, fourcc, VIDEO_FPS, (int(cap.get(3)), int(cap.get(4))))

# Function to calculate speed using Optical Flow
def calculate_speed(flow, factor_km=FACTOR_KM, latency_fps=LATENCY_FPS):
    magnitudes = np.sqrt(flow[:, :, 0] ** 2 + flow[:, :, 1] ** 2)
    average_speed = (np.mean(magnitudes) * factor_km * latency_fps)/1200
    return average_speed

while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, tracker='bytetrack.yaml', classes=[2,3,5,7])

        # Check if the results are not None and contain boxes
        if results is not None and len(results) > 0:
            # Get the boxes
            boxes = results[0].boxes.xywh.cpu()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Plot the tracks and calculate/display speeds using Optical Flow
            for box in boxes:
                x, y, w, h = box
                track_id = hash(box)
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point

                # Calculate speed using Optical Flow
                if len(track) > 1:
                    prev_pts = np.array(track[-2]).reshape(-1, 1, 2).astype(np.float32)
                    curr_pts = np.array(track[-1]).reshape(-1, 1, 2).astype(np.float32)

                    flow, status, _ = cv2.calcOpticalFlowPyrLK(
                        cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),
                        cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),
                        prev_pts,
                        None
                    )

                    speed = calculate_speed(flow)

                    # Display speed on the frame (corrected scaling)
                    cv2.putText(annotated_frame, f"Speed: {speed:.2f} km/h", (int(x), int(y) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Display the annotated frame
            # cv2.imshow("YOLOv8 Tracking", annotated_frame)
            out.write(annotated_frame)  # Save the frame to the output video

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video writer and close the display window
out.release()
cap.release()
cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.
0: 384x640 1 car, 1 truck, 29.6ms
Speed: 1.9ms preprocess, 29.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 29.7ms
Speed: 2.3ms preprocess, 29.7ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 31.7ms
Speed: 1.9ms preprocess, 31.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 31.6ms
Speed: 2.4ms preprocess, 31.6ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 30.8ms
Speed: 1.9ms preprocess, 30.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 31.9ms
Speed: 1.9ms preprocess, 31.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 32.7ms
Speed: 1.9ms preprocess, 32.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck

In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8x.pt')

# Open the video file
video_path = "/content/CornerSentinal/test_images/leftside.mp4"
cap = cv2.VideoCapture(video_path)

# Define the constants for speed calculation
VIDEO_FPS = 30
FACTOR_KM = 3.6
LATENCY_FPS = 15

# Store the track history
track_history = defaultdict(lambda: [])

# Define the output video file path with MP4 format
output_video_path = "/content/CornerSentinal/test_images/leftside_out.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use "mp4v" for H.264 compression
out = cv2.VideoWriter(output_video_path, fourcc, VIDEO_FPS, (int(cap.get(3)), int(cap.get(4))))

# Function to calculate speed using Optical Flow
def calculate_speed(flow, factor_km=FACTOR_KM, latency_fps=LATENCY_FPS):
    magnitudes = np.sqrt(flow[:, :, 0] ** 2 + flow[:, :, 1] ** 2)
    average_speed = (np.mean(magnitudes) * factor_km * latency_fps) / 1200
    return average_speed

# Placeholder function to simulate displaying binary data
def display_binary_code(frame, binary_code):
    binary_text = "Binary Code: " + ''.join(binary_code)
    cv2.putText(frame, binary_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, tracker='bytetrack.yaml', classes=[2, 3, 5, 7])

        # Check if the results are not None and contain boxes
        if results is not None and len(results) > 0:
            # Get the boxes
            boxes = results[0].boxes.xywh.cpu()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Plot the tracks and calculate/display speeds using Optical Flow
            for box in boxes:
                x, y, w, h = box
                track_id = hash(box)
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point

                # Calculate speed using Optical Flow
                if len(track) > 1:
                    prev_pts = np.array(track[-2]).reshape(-1, 1, 2).astype(np.float32)
                    curr_pts = np.array(track[-1]).reshape(-1, 1, 2).astype(np.float32)

                    flow, status, _ = cv2.calcOpticalFlowPyrLK(
                        cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),
                        cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),
                        prev_pts,
                        None
                    )

                    speed = calculate_speed(flow)

                    # Display speed on the frame (corrected scaling)
                    cv2.putText(annotated_frame, f"Speed: {speed:.2f} km/h", (int(x), int(y) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                    # Binary code generation based on conditions
                    binary_code = ['0'] * 6
                    # Condition 2: Class ID (assuming only one class is detected)

                    class_id = int(results[0].boxes.cls[0])  # Assuming class ID is at index 5
                    if class_id == 2:  # Adjust class ID for vehicles as needed
                        binary_code[1:4] = ['001']
                    elif class_id == 3:
                        binary_code[1:4] = ['010']
                    elif class_id == 5 or 7:
                        binary_code[1:4] = ['000'] # Class ID for vehicles
                    if speed > 60:
                        binary_code[5:] = '10'
                    # Display binary code on the frame
                    display_binary_code(annotated_frame, binary_code)

            # Display the annotated frame
            out.write(annotated_frame)  # Save the frame to the output video

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video writer and close the display window
out.release()
cap.release()
cv2.destroyAllWindows()


Streaming output truncated to the last 5000 lines.
0: 384x640 1 car, 31.7ms
Speed: 2.0ms preprocess, 31.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 31.9ms
Speed: 2.2ms preprocess, 31.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 31.9ms
Speed: 1.9ms preprocess, 31.9ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 30.4ms
Speed: 2.0ms preprocess, 30.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 31.2ms
Speed: 1.8ms preprocess, 31.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 30.9ms
Speed: 2.1ms preprocess, 30.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 33.1ms
Speed: 2.1ms preprocess, 33.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 31.7ms
Speed: 1.8ms preprocess, 31.7ms inference, 2.2ms postprocess pe

In [ ]:
!rm -rf '/content/leftside.mp4'

import os

# Set up NVIDIA GPU device for FFmpeg
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Convert hbfootage_out.mp4 to footage.mp4 using GPU-accelerated FFmpeg
os.system(f"ffmpeg -hwaccel cuda -i '/content/CornerSentinal/test_images/leftside_out.mp4' -vcodec hevc_nvenc '/content/leftside.mp4'")

0

In [ ]:
from google.colab import files
files.download('/content/leftside.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download('/content/rightside.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>